In [68]:
import pandas as pd
import unicodedata
import re
import nltk

In [69]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import nltk

# Loading data via google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
from huggingface_hub import notebook_login

In [72]:
notebook_login()

In [73]:
data_df = pd.read_csv(r"/content/drive/My Drive/data/dataset-metadata-with-lyrics.csv")
data_df = pd.DataFrame(data_df)
data_df

,song_id,song_name,Artist,Lyrics,Type
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේ\r\nඔබ දුරයි නෙත...,Calm
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේ\r\n\r\nබඹරු නටන හැන්දෑවේ…...,Calm
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,\r\nසමනළ මුදුනේ සිරිපද සිඹ සිඹ\r\nඋදා ඉරක් පාය...,Calm
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැව\r\nපමණි මට අද ශේ...,Calm
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා...\r\nඋදා හිරු එළියේ......,Calm
...,...,...,...,...,...
157,S 48,Dangakaaree Man,Indrani Wijebandara,දඟකාරී මං මද්දුම දෝණි\nඔබට අකීකරු නෑ අම්මේ\nදඟ...,Sad
158,S 49,Edaa Wage Adath,Somathilaka Jayamaha,එදා වාගේ අදත් පොද වැස්ස වැටෙනවා\r\nමල් පිපුණු ...,Sad
159,S 50,Paluwa Dewanath Karala,Gunadasa Kapuge,පාළුව දෙවනත් කරලා\r\nනාඩන් උලලේනෝ\r\nකාටත් නපු...,Sad
160,S 51,Pawena Nil Walawe,W. D. Amaradeva,පාවෙනා නිල් වලාවේ\r\nනදී තලාවේ\r\nරිදී ජලාසේ\r...,Sad


In [74]:
def lineBreakReplace(text):
    return text.replace("\r\n", " ")

In [75]:
data_df['Lyrics'] =data_df.apply(lambda x: lineBreakReplace(x.Lyrics), axis=1)

In [76]:
def removeEscapSequences(text):
    return "".join(ch for ch in text if unicodedata.category(ch)[0]!="C")

In [77]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [78]:

def remove_digits(text):
    return ''.join([i for i in text if not i.isdigit()])

In [79]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [80]:
def removeHtmlTags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

In [81]:

data_df['Lyrics'] = data_df.apply(lambda x: removeHtmlTags(x.Lyrics), axis=1)

In [82]:
data_df.head()

,song_id,song_name,Artist,Lyrics,Type
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේ ඔබ දුරයි නෙතට ම...,Calm
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේ බඹරු නටන හැන්දෑවේ… සැදෑ ...,Calm
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,සමනළ මුදුනේ සිරිපද සිඹ සිඹ උදා ඉරක් පායයි සැන...,Calm
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැව පමණි මට අද ශේෂ ව...,Calm
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා... උදා හිරු එළියේ... අළ...,Calm


In [83]:
def removeSpecialCharacters(sentence,keep_apostrophes=True):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
        return filtered_sentence
    return sentence

In [84]:
def replace_full_stops(sentence):
    return sentence.replace("."," ")

In [85]:
data_df['Lyrics'] = data_df.apply(lambda x: removeSpecialCharacters(x.Lyrics), axis=1)
data_df['Lyrics'] = data_df.apply(lambda x: replace_full_stops(x.Lyrics), axis=1)

In [86]:
def remove_english_letters(sentence):
    non_english_sentence = re.sub(r'[a-zA-Z]', '', sentence)
    return non_english_sentence

In [87]:
data_df['Lyrics']  = data_df['Lyrics'] .apply(lambda x: remove_english_letters(x))

In [88]:
lines = list(open('/content/drive/My Drive/data/stop-words-sinhala.txt',encoding="utf8"))
sinhala_stop_words = [re.sub("\n","",x) for x in lines]

In [89]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [90]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [91]:
data_df['token_list']  = data_df['Lyrics'] .apply(lambda x: tokenize_text(x))

In [92]:
def remove_sinhala_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in sinhala_stop_words]
    return filtered_tokens

In [93]:

#data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_sinhala_stop_words(x.token_list), axis=1)

In [94]:
def remove_two_letter_words(tokenized_words):
    # Use list comprehension to filter out words with length 2
    filtered_words = [word for word in tokenized_words if len(word) > 3]
    return filtered_words

In [95]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_two_letter_words(x.token_list), axis=1)

In [96]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [97]:
list_of_words = ' '.join(data_df['pre_processed_text']).split()

In [98]:
from collections import Counter
import numpy as np
most_common_words = [word for word, count in Counter(list_of_words).items() if count in np.arange(50,100)]
unique_words = [word for word, count in Counter(list_of_words).items() if count in [1,2,3]]
all_words_to_remove = most_common_words+unique_words;
def remove_most_common_unique_words(tokens):
    filtered_tokens = [token for token in tokens if token not in all_words_to_remove]
    return filtered_tokens

In [99]:
data_df['token_list_after_common_words'] = data_df.apply(lambda x: remove_most_common_unique_words(x.token_list_after_sw), axis=1)

In [100]:
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_common_words), axis=1)

In [101]:
# list_of_words = ' '.join(data_df['pre_processed_text']).split()
# Counter(list_of_words)

In [102]:
data_df['word_count'] = data_df['pre_processed_text'].str.split().str.len()
#data_df.head(40)

In [103]:
def total_unique_words(words):
    return nltk.FreqDist(words)

print("Total number of unique words",total_unique_words(list_of_words))

In [104]:
def total_words(words):
    return(len(words))
print("Total number of words",total_words(list_of_words))

In [105]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [106]:
#X= data_df['pre_processed_text']
data_df["final_data"] = "__label__"+data_df['Type'] + " " + data_df["pre_processed_text"]
#data_df
#y=label_encoder.fit_transform(data_df['Type'])

In [107]:
! pip install -U accelerate
! pip install -U transformers

In [108]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [109]:
!pip install datasets

In [110]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [111]:
data_df['label'] =labelencoder.fit_transform(data_df['Type'])

In [112]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
def tokenize_function(examples):
    return tokenizer(examples["pre_processed_text"], padding='max_length', truncation=True, max_length=180)

In [113]:
from datasets import Dataset
h_dataset = Dataset.from_pandas(data_df)

In [114]:
h_dataset

Dataset({
    features: ['song_id', 'song_name', 'Artist', 'Lyrics', 'Type', 'token_list', 'token_list_after_sw', 'pre_processed_text', 'token_list_after_common_words', 'word_count', 'final_data', 'label'],
    num_rows: 162
})

In [115]:
tokenized_dataset = h_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

Dataset({
    features: ['song_id', 'song_name', 'Artist', 'Lyrics', 'Type', 'token_list', 'token_list_after_sw', 'pre_processed_text', 'token_list_after_common_words', 'word_count', 'final_data', 'label', 'input_ids', 'attention_mask'],
    num_rows: 162
})

In [116]:
label_names = ["Calm", "Happy", "Sad"]
id2label = {idx:label for idx, label in enumerate(label_names)}
id2label

{0: 'Calm', 1: 'Happy', 2: 'Sad'}

In [117]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'Calm': 0, 'Happy': 1, 'Sad': 2}

In [118]:
#train, test = train_test_split(tokenized_dataset ,test_size=0.2, random_state=42)

In [119]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels, label2id=label2id, id2label=id2label)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [120]:
dataset=tokenized_dataset.shuffle().train_test_split(test_size=0.3)

In [121]:
 model_checkpoint = "xlm-roberta-base"

In [122]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_true, y_pred)
# print(f"Accuracy: {accuracy * 100:.2f}%")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
   # print(predictions)
    predictions = np.argmax(predictions, axis=1)
    print(labels)
    print(predictions)
    print("=============================")
    return {"Accuracy": accuracy_score(labels, predictions) }

In [123]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]
batch_size = 64
num_train_epochs = 40
logging_steps = len(dataset["train"]) // (batch_size * num_train_epochs)

args = TrainingArguments(
    output_dir=f"{model_name}-sinhala-song-cls",
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=40,
    weight_decay=0.01,
    save_total_limit = 2,
    load_best_model_at_end=True,
    #logging_steps=logging_steps,
    push_to_hub=True,
  #  load_best_model_at_end= True,
)

In [124]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [125]:
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.105700,1.098286,0.346939
2,1.092600,1.097962,0.346939
3,1.095800,1.096779,0.346939
4,1.098100,1.096107,0.346939
5,1.089500,1.091203,0.326531
6,1.051100,1.066476,0.510204
7,0.982400,1.035697,0.510204
8,0.960300,1.013716,0.530612
9,0.952300,1.020724,0.510204
10,0.931300,1.033561,0.510204


TrainOutput(global_step=80, training_loss=0.5906762175261975, metrics={'train_runtime': 463.2853, 'train_samples_per_second': 9.756, 'train_steps_per_second': 0.173, 'total_flos': 418103665358400.0, 'train_loss': 0.5906762175261975, 'epoch': 40.0})

In [126]:
# from google.colab import files
# files.download('/content/drive/MyDrive/roberta-base-sinhala-song-cls')

In [127]:
model.push_to_hub("xlm-roberta-base-sinhala-song-cls")

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/malinda135/xlm-roberta-base-sinhala-song-cls/commit/d2949be04ee11256cd2d9028f0ecd20c02f3e90b', commit_message='Upload XLMRobertaForSequenceClassification', commit_description='', oid='d2949be04ee11256cd2d9028f0ecd20c02f3e90b', pr_url=None, pr_revision=None, pr_num=None)